In [3]:
import geopandas as gpd 
import shapely 
import folium 
import pandas as pd 
import re

In [4]:
def  format_longitude(coordinates):
  matches = re.findall(r'\d+(?:\.\d+)?', coordinates)
  deg = float(matches[0])
  sec = float(matches[1])
  mil = float(matches[2])
  return -(deg + sec / 60 + mil / 3600)
def  format_latitude(coordinates):
  matches = re.findall(r'\d+(?:\.\d+)?', coordinates)
  deg = float(matches[0])
  sec = float(matches[1])
  mil = float(matches[2])
  return deg + sec / 60 + mil / 3600

In [9]:
file_path = "ZONE DAKHLA (1).xlsx"
data = pd.read_excel(file_path)
data.drop("Unnamed: 0" , axis =1 ).dropna()

,Zone A,Point 1,Point 2,Point 3,Point 4
0,LAT,"22°16'07""","21°56'34""","21°56'34""","22°16'07"""
1,LONG,"15°49'13""","15°49'13""","16°20'51""","16°20'51"""
4,Zone B,Point 4,Point 5,Point 6,Point 8
5,LAT,"22°55'04""","22°55'04""","22°16'07""","22°16'07"""
6,LONG,"15°28'07""","16°06'00""","16°06'00""","15°28'07"""
9,Zone C,Point 9,Point 10,Point 11,Point 12
10,LAT,"22°04'46""","22°04'46""","22°55'04""","22°55'04"""
11,LONG,"15°07'01""","15°17'34""","15°17'34""","15°07'01"""


In [12]:
data.drop("Unnamed: 0" , axis =1 ).dropna().transpose()

,0,1,4,5,6,9,10,11
Zone A,LAT,LONG,Zone B,LAT,LONG,Zone C,LAT,LONG
Point 1,"22°16'07""","15°49'13""",Point 4,"22°55'04""","15°28'07""",Point 9,"22°04'46""","15°07'01"""
Point 2,"21°56'34""","15°49'13""",Point 5,"22°55'04""","16°06'00""",Point 10,"22°04'46""","15°17'34"""
Point 3,"21°56'34""","16°20'51""",Point 6,"22°16'07""","16°06'00""",Point 11,"22°55'04""","15°17'34"""
Point 4,"22°16'07""","16°20'51""",Point 8,"22°16'07""","15°28'07""",Point 12,"22°55'04""","15°07'01"""


In [102]:
# create data frame for each point in the data frame 
d = data.drop("Unnamed: 0" , axis =1 ).dropna().transpose()
d

,0,1,4,5,6,9,10,11
Zone A,LAT,LONG,Zone B,LAT,LONG,Zone C,LAT,LONG
Point 1,"22°16'07""","15°49'13""",Point 4,"22°55'04""","15°28'07""",Point 9,"22°04'46""","15°07'01"""
Point 2,"21°56'34""","15°49'13""",Point 5,"22°55'04""","16°06'00""",Point 10,"22°04'46""","15°17'34"""
Point 3,"21°56'34""","16°20'51""",Point 6,"22°16'07""","16°06'00""",Point 11,"22°55'04""","15°17'34"""
Point 4,"22°16'07""","16°20'51""",Point 8,"22°16'07""","15°28'07""",Point 12,"22°55'04""","15°07'01"""


In [103]:
# restore the dataframe in correct form 
d = d.transpose()
d

,Zone A,Point 1,Point 2,Point 3,Point 4
0,LAT,"22°16'07""","21°56'34""","21°56'34""","22°16'07"""
1,LONG,"15°49'13""","15°49'13""","16°20'51""","16°20'51"""
4,Zone B,Point 4,Point 5,Point 6,Point 8
5,LAT,"22°55'04""","22°55'04""","22°16'07""","22°16'07"""
6,LONG,"15°28'07""","16°06'00""","16°06'00""","15°28'07"""
9,Zone C,Point 9,Point 10,Point 11,Point 12
10,LAT,"22°04'46""","22°04'46""","22°55'04""","22°55'04"""
11,LONG,"15°07'01""","15°17'34""","15°17'34""","15°07'01"""


In [109]:
list_d = []
for i in range(0 , len(d) , 3 ): 
    data_i = d[i : i+2]
    data_i.index = ["lat" , "lon"]
    data_i = data_i.transpose()
    data_i = data_i.drop(data_i.index[0] , axis = 0)
    data_i["lati"] = data_i.lat.apply(format_latitude)
    data_i["longi"] = data_i.lon.apply(format_longitude)
    list_d.append(data_i)

In [119]:
list_d[0].lati.ilco



Point 1    22.268611
Point 2    21.942778
Point 3    21.942778
Point 4    22.268611
Name: lati, dtype: float64

In [124]:
# form the data frame of zones 
polygons = []
zone     = []
for i in range(len(list_d)): 
    poly = []
    for j in range(len(list_d[i])):
        poly.append((list_d[i].longi.iloc[j] ,list_d[i].lati.iloc[j]))
    polygons.append(shapely.Polygon(poly))
    zone.append("zone{}".format(i))
        
        

In [132]:
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [134]:
zones = gpd.GeoDataFrame({"name" : zone} , geometry = polygons , crs = "EPSG:4326")
zones.to_file("zones_ETUDE.KML" , driver = "KML")

In [135]:
# export each zone in its specified shp file 
zones 

,name,geometry
0,zone0,"POLYGON ((-15.82028 22.26861, -15.82028 21.942..."
1,zone1,"POLYGON ((-15.46861 22.91778, -16.10000 22.917..."
2,zone2,"POLYGON ((-15.11694 22.07944, -15.29278 22.079..."


In [142]:
index = 1
for i in range(len(zones)) : 
    zones.iloc[[i]].to_file("zone{}.shp".format(i))